# Crawling [bezdim.org](https://bezdim.org/signali/reports)

### Import Scrapy and install if missing

In [1]:
try:
    import scrapy
except:
    import sys
    !conda install --yes --prefix {sys.prefix} -c conda-forge scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

#### Other imports

In [2]:
from time import time
from os.path import join
import pandas as pd

timestamp = int(time())

Currently not using the custom pipeline

In [3]:
import json
import codecs


class JsonWriterPipeline:

    def open_spider(self, spider):
        self.file = codecs.open('reportresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

### Helper functions

In [4]:
class ParseUtils:

    @staticmethod
    def parse_description(raw_description):
        striped = [d.strip() for d in raw_description if d.strip()]
        return " ".join(striped)
    
    @staticmethod
    def parse_time_location(when_where_element):
        selectors = when_where_element.xpath('.//span')
        if len(selectors) < 1:
            return ('N/A Data', 'N/A Location')
        
        date = selectors[0].css('span.r_date::text').extract_first(default='N/A Date')
        location = 'N/A Location'
        if len(selectors) > 1:
            location = selectors[1].css('span.r_location::text').extract_first(default='N/A Location')
    
        return date, location
    
    @staticmethod
    def parse_categories(categories_element):
        '''
        Can have multiple categories
        https://bezdim.org/signali/reports/view/10099
        '''
        selector = categories_element.xpath('.//p//a')
        categories = selector.css('a::text').extract()
        
        return [cat.strip() for cat in categories]
    

In [5]:
import logging
from scrapy import Spider
from scrapy.http import Request

class BezDimSpider(Spider):
    name = "bezdim"
    max_page_count = 5
    start_urls = [
        'https://bezdim.org/signali/reports'
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
#         'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},
        'FEED_FORMAT':'json',
        'FEED_URI': join('reports_data','report-result-{timestamp}.json'.format(timestamp=timestamp))
    }
    
    def start_requests(self):
        for i in range(1, self.max_page_count):
            yield Request('{url}/fetch_reports?page={page_id}'.
                          format(url=self.start_urls[0], page_id=i),
                    callback=self.process_page)
    
    def process_page(self, response):
        for link in response.css('a.r_title'):
            url = link.xpath('@href').extract_first()
            yield Request(url, callback=self.parse)
    
    def parse(self, report):
        title = report.css('h1.report-title::text').extract_first(default='').strip()
        
        raw_desc = report.css('div.report-description-text::text').extract()
        description = ParseUtils.parse_description(raw_desc)
        
        date, location = ParseUtils.parse_time_location(report.css('p.report-when-where'))
        
        categories = ParseUtils.parse_categories(report.css('div.report-category-list'))
        

        yield {
            'title': title,
            'description': description,
            'date': date,
            'location': location,
            'categories': categories
        }
        
#             description = report.css('div.r_description::text').extract_first().strip()
#             address = report.css('p.bd_location::text').extract_first().strip()
            
            

In [6]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(BezDimSpider)
process.start()

2019-01-25 22:04:45 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: scrapybot)
2019-01-25 22:04:45 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 18.9.0, Python 3.7.1 (default, Dec 14 2018, 19:28:38) - [GCC 7.3.0], pyOpenSSL 18.0.0 (OpenSSL 1.1.1a  20 Nov 2018), cryptography 2.4.2, Platform Linux-4.19.14-1-MANJARO-x86_64-with-arch-Manjaro-Linux
2019-01-25 22:04:45 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'reports_data/report-result-1548446685.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


In [7]:
import pandas as pd

a = pd.read_json(
    join('reports_data','report-result-{timestamp}.json'.format(timestamp=timestamp)))

In [8]:
a

,categories,date,description,location,title
0,[заведение за хранене и развлечение],10:30 Dec 1 2018,"име обект: МОЛ ВАРНА вид обект: заведение ""ПЛЕ...","град Варна, бул. „Владислав Варненчик“ 186","Пушене в заведение ""ПЛЕЙГРАУНД"" МОЛ ВАРНА, гра..."
1,[заведение за хранене и развлечение],01:30 Dec 3 2018,"В кафе ""Зограф""/съставна част от семеен х-л ""З...","град Трявна, ул. ""П.Р.Славейков"" № 1","Незаконно обособена зона за пушене,тровеща неп..."
2,[заведение за хранене и развлечение],08:18 Nov 29 2018,"Бяха донесени чашки със салфетки, в които да с...","град Пловдив, бул. ""Никола Вапцаров"" 50","Пушене на закрито в ресторант Марс, град Пловдив"
3,[заведение за хранене и развлечение],03:21 Dec 5 2018,Пушене ежедневно в кафене на бензиностанция-га...,"село Ягодово, област Пловдив, ул.Васил Левски...","Пушене в бензиностанция-газстанция на закрито,..."
4,[заведение за хранене и развлечение],07:13 Nov 5 2018,"В ресторант ""Немо"" гр. Варна, пристанище Варн...","град Варна, Морска гара","Заведение с тютюнопушене, град Варна"
5,[заведение за хранене и развлечение],07:16 Nov 7 2018,Пуши се на закрито в заведението The river в г...,"град Русе, ул. Чавдар Войвода 33 /пресечна с б...","Пушене на закрито в заведение за хранене, град..."
6,[заведение за хранене и развлечение],09:18 Nov 27 2018,В ресторант Тео Елит в град Пловдив се пуши * ...,"град Пловдив, ул. ""Съединение"", срещу бл. 151","Нарушение на забрана за тютюнопушене, град Пло..."
7,[заведение за хранене и развлечение],00:43 Nov 11 2018,"В ресторант Родопска къща в Студентски град, С...","град София, ул. ""8-ми декември"", блок 21","Пушене в ресторант Родопска къща, град София"
8,[работно място],08:47 Nov 12 2018,Позволява се пушенето на закрито в снимачния к...,"Нови хан, община Елин Пелин, Софийска област","Пушене на работно място, Нови хан, община Елин..."
9,[приют и център за временно настаняване],10:04 Nov 12 2018,В предаването Биг Брадър се пуши в затворени о...,"Софийса област, община Елин пелин, Нови Хан, с...","Пушене в студиото на Биг Брадър, Нови хан, общ..."
